In [1]:
import pandas as pd
import re
from utils import mahalanobis_algorithm, valid_params, prepare_params, load_properties

## Leitura e tratamento dos dados

In [2]:
df_imoveis = load_properties()

## Base utilizada

In [3]:
df_imoveis.head()

,id,value,location,price_suggested,categoria,area,rooms,bathrooms,garages,district_name
3,6074590677dee513e95c7f1f,200001.0,"{'lat': -10.9299886, 'lng': -37.0830428}",199906.0,Apartamentos,61.0,2.0,2.0,1.0,Jabotiana
6,6074591277dee513e95c7f22,430000.0,"{'lat': -10.9926775, 'lng': -37.0552954}",369970.0,Apartamentos,0.0,2.0,2.0,1.0,Atalaia
11,6074592477dee513e95c7f27,480000.0,"{'lat': -10.9282507, 'lng': -37.056637}",554047.0,Apartamentos,140.0,4.0,4.0,2.0,Salgado Filho
12,6074592877dee513e95c7f28,95000.0,"{'lat': -10.8726535, 'lng': -37.0763127}",103214.0,Apartamentos,45.0,2.0,1.0,1.0,Lamarão
13,6074592c77dee513e95c7f29,940019.0,"{'lat': -10.9472468, 'lng': -37.0730823}",940008.0,Apartamentos,140.0,3.0,4.0,2.0,Grageru


## Bairros válidos

In [4]:
AVAILABLE_DISTRICTS = ['Jabotiana', 'Atalaia', 'Salgado Filho', 'Lamarão', 'Grageru',
                       'Luzia', 'Jardins', 'Farolândia', 'Coroa do Meio', 'Ponto Novo',
                       'Suíssa', 'Centro', 'São Conrado', 'Inácio Barbosa', 'São José',
                       'Zona de Expansão (Mosqueiro)', 'Treze de Julho',
                       'Dezoito do Forte', 'Aeroporto', 'Zona de Expansão (Aruana)',
                       'Siqueira Campos', 'Pereira Lobo', 'Jardim Centenário',
                       'Santa Maria', 'Dom Luciano', 'Olaria', 'Santos Dumont', 'América',
                       'Santo Antônio', 'Getúlio Vargas', 'Cirurgia', 'Industrial',
                       "Porto D'Antas", '18 do Forte', 'José Conrado de Araújo',
                       'Cidade Nova', 'Zona de Expansão (Robalo)', '13 De Julho',
                       'Palestina', 'Soledade', 'Japãozinho']

## Preferências do Usuário 

In [10]:
params = {
    "value": 200000,        # Valor em Reais (R$)
    "rooms": 3,             # Número de quartos
    "area": 80,             # Área do imóvel em metros quadrados
    "bathrooms": 2,         # Número de banheiros do imóvel
    "garages": 1,           # Número de garagens do imóvel
    "district": "Atalaia"   # Bairro onde se localiza o imóvel
}

## Validar parametros 

## Preparar os parametros

In [12]:
from utils import prepare_params

prepared_params, error = prepare_params(params)

In [13]:
prepared_params

{'value': 200000.0,
 'rooms': 3.0,
 'area': 80.0,
 'bathrooms': 2.0,
 'garages': 1.0,
 'district': 'Atalaia'}

## Alterar a base de acordo com os parametros

In [14]:
from utils import handle_dataframe_values

df, params_to_algorithm = handle_dataframe_values(df_imoveis, prepared_params)
df

,id,value,location,price_suggested,categoria,area,rooms,bathrooms,garages,district_name,distance,district
3,6074590677dee513e95c7f1f,200001.0,"{'lat': -10.9299886, 'lng': -37.0830428}",199906.0,Apartamentos,61.0,2.0,2.0,1.0,Jabotiana,7.193320,0.0
6,6074591277dee513e95c7f22,430000.0,"{'lat': -10.9926775, 'lng': -37.0552954}",369970.0,Apartamentos,0.0,2.0,2.0,1.0,Atalaia,0.547648,1.0
11,6074592477dee513e95c7f27,480000.0,"{'lat': -10.9282507, 'lng': -37.056637}",554047.0,Apartamentos,140.0,4.0,4.0,2.0,Salgado Filho,6.626499,0.0
12,6074592877dee513e95c7f28,95000.0,"{'lat': -10.8726535, 'lng': -37.0763127}",103214.0,Apartamentos,45.0,2.0,1.0,1.0,Lamarão,13.009464,0.0
13,6074592c77dee513e95c7f29,940019.0,"{'lat': -10.9472468, 'lng': -37.0730823}",940008.0,Apartamentos,140.0,3.0,4.0,2.0,Grageru,4.999925,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5073,608ecdded17e5ccb234072ca,156000.0,"{'lat': -10.9268471, 'lng': -37.0502469}",185828.0,Apartamentos,0.0,2.0,0.0,0.0,São José,6.781647,0.0
5074,608ecde3d17e5ccb234072cb,220000.0,"{'lat': -10.9702822, 'lng': -37.057117}",215008.0,Apartamentos,78.0,3.0,1.0,1.0,Farolândia,2.008431,0.0
5075,608ecde6d17e5ccb234072cc,280050.0,"{'lat': -10.9425999, 'lng': -37.0677036}",280029.0,Apartamentos,123.0,4.0,3.0,1.0,Luzia,5.265825,0.0
5077,608ecdf2d17e5ccb234072ce,0.0,"{'lat': -10.9484887, 'lng': -37.0760862}",186360.0,Apartamentos,64.0,3.0,2.0,1.0,Inácio Barbosa,5.029325,0.0
